In [1]:
import time
import copy
import os
import sys

import numpy as np
import pandas as pd
import geopandas as gpd
import multiprocess

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/%autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = None  # display all columns

In [2]:
# need to set this environmental path everytime you moves the project root folder
os.environ['project_root'] = '/Users/geekduck/Dropbox/gt_survey'

sys.path.append(
    os.environ['project_root']
)

In [35]:
from carpoolsim.basic_settings import *

from carpoolsim.prepare_input import (
    get_shortest_paths,
    TrafficNetwork
)

In [4]:
# load traffic network data
taz_centriod_nodes = gpd.read_file(os.environ["taz"])
df_nodes_raw = gpd.read_file(os.environ['network_nodes'])
df_links_raw = gpd.read_file(os.environ['network_links'])

In [5]:
taz_centriod_nodes["taz_id"] = taz_centriod_nodes["OBJECTID"]

In [7]:
# init object
traffic_network = TrafficNetwork(
    network_links=df_links_raw.copy(),
    network_nodes=df_nodes_raw.copy(),
    tazs=taz_centriod_nodes.copy(),
)

In [8]:
traffic_network.get_taz_id_list()
traffic_network.convert_abm_links()
traffic_network.build_network()

In [9]:
taz_lst = traffic_network.prepare_taz_lists(chunk_size=100)

In [12]:
network_dict = traffic_network.network_dict
destination_lst = traffic_network.tazs_ids
taz_lst = traffic_network.prepare_taz_lists(chunk_size=100)
task_inputs = [
    (network_dict, destination_lst, taz)
    for taz in taz_lst
]

In [13]:
network_dict["DG"]

In [14]:
len(destination_lst)

5873

In [15]:
t0 = time.perf_counter()
results = None
with multiprocess.Pool(NUM_PROCESSES) as pool:
    results = pool.starmap(get_shortest_paths, task_inputs)

d1 = time.perf_counter() - t0
print(f'It takes {d1/60:.1f} minutes to finish the run')

Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 101 tazs
Finished searching 1

## Store results in a database

In [44]:
# create database connection
from carpoolsim.database.prepare_database import batch_store_from_lst

from sqlalchemy import create_engine

In [45]:
engine = create_engine(DB_URL)

with engine.connect() as connection:
    print('successfully connected')

successfully connected


In [ ]:
batch_store_from_lst(results, DB_URL)

sqlite engine created
